In [ ]:
#yolo rough

In [ ]:
#conv lastm code Arch 
#make mobilenet+use arch 
#maek 2streamacrch 
#train on bounding box
#train on full frame
#combine output of both
#this is called hybrid approach twostram or multistram architecture

Hybrid approach (common in research)
Feed both: a cropped bounding box stream (for subject-focused motion) and a full-frame stream (for context).

This is called a two-stream or multi-stream architecture — common in action recognition models like I3D, SlowFast, or VideoMAE.

1)Example shapes for batch size B:

cropped_seq  → [B, T, 3, H, W]
full_seq     → [B, T, 3, H, W]
label        → [B]

2)Feed both streams at once:
outputs = model(cropped_seq, full_seq)  # both go through their own MobileNet+ConvLSTM
loss = criterion(outputs, labels)


In [2]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset,TensorDataset
import cv2
import os
import random
import numpy as np
import pandas as pd

In [ ]:
'''  
Raw dataset

YOLO detection

Save processed (cropped) dataset

Train/test split

On-the-fly augmentations during training
'''

In [ ]:
df = pd.read_csv("dog_dataset_full.csv")
df['label'] = df['label'].map({'aggressive': 1, 'notaggressive': 0}) #encoding labels

if 'action' in df.columns:
    df = pd.get_dummies(df, columns=['action']) # One-hot encoding for 'action' column


col_names = [col for col in df.columns if col not in ['video_path', 'label']] #collecting col names

#convert to pytorch tensor
X = torch.tensor(df[col_names].values, dtype=torch.float32)
y = torch.tensor(df['label'].values, dtype=torch.long)

#df[col_names].values #extract features values as numpy array

array([[ True, False, False, ..., False, False, False],
       [ True, False, False, ..., False, False, False],
       [ True, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False,  True, False],
       [False, False, False, ..., False,  True, False],
       [False, False, False, ..., False, False,  True]], shape=(294, 16))

In [5]:
df.head()

,video_path,label,action_barks_clips,action_chasing_toy,action_chewing,action_digging,action_drinking,action_eating_food,action_growling_clips,action_howling,action_jump,action_panting,action_playing,action_rolling,action_sitting,action_sleeping,action_teeth_chattring,action_wagging_tail
0,C:\Users\aliza\Desktop\og_behaviour\dataset_2\...,1.0,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,C:\Users\aliza\Desktop\og_behaviour\dataset_2\...,1.0,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,C:\Users\aliza\Desktop\og_behaviour\dataset_2\...,1.0,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,C:\Users\aliza\Desktop\og_behaviour\dataset_2\...,1.0,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,C:\Users\aliza\Desktop\og_behaviour\dataset_2\...,1.0,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [9]:

from sklearn.model_selection import train_test_split

# Train-test split with stratification
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [11]:


dataset = TensorDataset(X, y)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

print(f"Dataset size: {len(dataset)}")
print(f"Number of batches: {len(dataloader)}")
print(f"Feature shape: {X.shape}")
print(f"Label shape: {y.shape}")


Dataset size: 294
Number of batches: 19
Feature shape: torch.Size([294, 16])
Label shape: torch.Size([294])


In [ ]:
##different 

import os
import cv2
import torch
import numpy as np
import torch.nn as nn
from torch.utils.data import Dataset
import torchvision.models as models
import torchvision.transforms as T

import pandas as pd
from sklearn.model_selection import train_test_split

# read csv
df = pd.read_csv("dog_dataset_full.csv")

# map labels to int
df['label'] = df['label'].map({'aggressive': 1, 'notaggressive': 0})

train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

print(f"Train size: {len(train_df)}, Val size: {len(val_df)}")


# MobileNetV2 feature extractor
mobilenet = models.mobilenet_v2(pretrained=True)
feature_extractor = nn.Sequential(*list(mobilenet.children())[:-1])  # remove classifier
feature_extractor.eval()

class VideoFeatureDataset(Dataset):
    def __init__(self, df, root_dir, transform=None, max_seq_len=32):
        self.df = df
        self.root_dir = root_dir
        self.transform = transform
        self.max_seq_len = max_seq_len

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        video_path = os.path.join(self.root_dir, row["video_path"])
        label = row["label"]

        frames = self.load_video(video_path)
        frames = self.pad_or_truncate(frames, self.max_seq_len)

        return frames, torch.tensor(label, dtype=torch.long)

    def load_video(self, path):
        cap = cv2.VideoCapture(path)
        frames = []
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = cv2.resize(frame, (224,224))

            frame = torch.tensor(frame, dtype=torch.float32).permute(2,0,1)/255.0
            if self.transform:
                frame = self.transform(frame)

            # extract features
            with torch.no_grad():
                f = feature_extractor(frame.unsqueeze(0))  # (1,1280,7,7)
                f = torch.mean(f, dim=[2,3]).squeeze(0)   # (1280,)
            frames.append(f)
        cap.release()
        return torch.stack(frames) if frames else torch.zeros((0,1280))

    def pad_or_truncate(self, frames, max_len):
        length = frames.shape[0]
        if length >= max_len:
            return frames[:max_len]
        else:
            pad = torch.zeros((max_len - length, frames.shape[1]))
            return torch.cat([frames, pad], dim=0)
        

train_transforms = T.Compose([
    T.RandomHorizontalFlip(),
    T.ColorJitter(brightness=0.2, contrast=0.2)
])

train_dataset = VideoFeatureDataset(train_df, root_dir="videos", transform=train_transforms, max_seq_len=32)
val_dataset   = VideoFeatureDataset(val_df,   root_dir="videos", transform=None, max_seq_len=32)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader   = torch.utils.data.DataLoader(val_dataset, batch_size=8, shuffle=False)

class LSTMClassifier(nn.Module):
    def __init__(self, input_dim=1280, hidden_dim=512, num_layers=2, num_classes=2):
        super(LSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_dim*2, num_classes)  # *2 for bidirectional

    def forward(self, x):
        # x: (batch, seq_len, feature_dim)
        out, _ = self.lstm(x)
        out = out[:, -1, :]  # last timestep
        return self.fc(out)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = LSTMClassifier(input_dim=1280, hidden_dim=512, num_layers=2, num_classes=2).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

for epoch in range(10):
    model.train()
    for features, labels in train_loader:
        features, labels = features.to(device), labels.to(device)
        outputs = model(features)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}: Train loss {loss.item():.4f}")


In [ ]:
#dataloader
#dataset should return paired sequences
class TwoStreamDataset(torch.utils.data.Dataset):
    def __init__(self, crop_videos, full_videos, labels):
        self.crop_videos = crop_videos
        self.full_videos = full_videos
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        crop_seq = self.crop_videos[idx]  # Tensor [T, C, H, W]
        full_seq = self.full_videos[idx]  # Tensor [T, C, H, W]
        label = self.labels[idx]
        return crop_seq, full_seq, label


In [ ]:
#conv lstm code
import torch
import torch.nn as nn

class ConvLSTMCell(nn.Module):
    def __init__(self, input_dim, hidden_dim, kernel_size, bias=True):
        super().__init__()
        self.hidden_dim = hidden_dim
        padding = kernel_size // 2

        self.conv = nn.Conv2d(
            in_channels=input_dim + hidden_dim,
            out_channels=4 * hidden_dim,
            kernel_size=kernel_size,
            padding=padding,
            bias=bias
        )

    def forward(self, x, hidden):
        h_cur, c_cur = hidden
        combined = torch.cat([x, h_cur], dim=1)  # [B, C, H, W]
        conv_output = self.conv(combined)
        cc_i, cc_f, cc_o, cc_g = torch.chunk(conv_output, 4, dim=1)

        i = torch.sigmoid(cc_i)
        f = torch.sigmoid(cc_f)
        o = torch.sigmoid(cc_o)
        g = torch.tanh(cc_g)

        c_next = f * c_cur + i * g
        h_next = o * torch.tanh(c_next)
        return h_next, c_next

    def init_hidden(self, batch_size, image_size):
        H, W = image_size
        return (torch.zeros(batch_size, self.hidden_dim, H, W, device='cuda'),
                torch.zeros(batch_size, self.hidden_dim, H, W, device='cuda'))


In [ ]:
#mobilenet+convlstm
import torchvision.models as models

class MobileNetConvLSTM(nn.Module):
    def __init__(self, hidden_dim=256, num_classes=10):
        super().__init__()
        
        mobilenet = models.mobilenet_v2(pretrained=True)
        self.feature_extractor = mobilenet.features  # keep spatial feature maps
        feature_channels = 1280  # last layer channels
        
        self.convlstm = ConvLSTMCell(input_dim=feature_channels, hidden_dim=hidden_dim, kernel_size=3)
        
        # Classifier
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x_seq):
        """
        x_seq: [B, T, C, H, W]
        """
        b, t, c, h, w = x_seq.shape
        h_state, c_state = self.convlstm.init_hidden(b, (h//32, w//32))  # MobileNet downsamples by 32

        for frame in range(t):
            features = self.feature_extractor(x_seq[:, frame])  # [B, 1280, H', W']
            h_state, c_state = self.convlstm(features, (h_state, c_state))

        pooled = self.pool(h_state).view(b, -1)
        return self.fc(pooled)


In [ ]:
#2stream
class TwoStreamMobileNetConvLSTM(nn.Module):
    def __init__(self, hidden_dim=256, num_classes=10):
        super().__init__()
        self.stream1 = MobileNetConvLSTM(hidden_dim, num_classes)
        self.stream2 = MobileNetConvLSTM(hidden_dim, num_classes)
        self.fc_final = nn.Linear(num_classes*2, num_classes)

    def forward(self, seq1, seq2):
        out1 = self.stream1(seq1)  # cropped
        out2 = self.stream2(seq2)  # full frame
        combined = torch.cat([out1, out2], dim=1)
        return self.fc_final(combined)


In [ ]:
#trianing loop
device = "cuda" if torch.cuda.is_available() else "cpu"
model = MobileNetConvLSTM(hidden_dim=256, num_classes=10).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

for epoch in range(10):
    model.train()
    for batch in train_loader:
        videos, labels = batch  # videos: [B, T, C, H, W]
        videos, labels = videos.to(device), labels.to(device)

        outputs = model(videos)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1} - Loss: {loss.item():.4f}")


In [ ]:
#other way
import torch
import torch.nn as nn
import torchvision.models as models

# ConvLSTM module (simplified version)
class ConvLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, kernel_size=(3,3), num_layers=1):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)

    def forward(self, x):
        # x shape: [B, T, features]
        output, _ = self.lstm(x)
        return output[:, -1, :]  # last time step

# Stream network: MobileNetV2 → ConvLSTM
class StreamNet(nn.Module):
    def __init__(self, mobilenet_weights=True):
        super().__init__()
        mobilenet = models.mobilenet_v2(pretrained=mobilenet_weights)
        mobilenet.classifier = nn.Identity()  # remove final classification layer
        self.cnn = mobilenet
        self.temporal = ConvLSTM(input_dim=1280, hidden_dim=512)

    def forward(self, x):
        # x shape: [B, T, C, H, W]
        B, T, C, H, W = x.size()
        feats = []
        for t in range(T):
            f = self.cnn(x[:, t])  # [B, 1280]
            feats.append(f)
        feats = torch.stack(feats, dim=1)  # [B, T, 1280]
        return self.temporal(feats)

# Two-stream fusion network
class TwoStreamNet(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.stream1 = StreamNet()
        self.stream2 = StreamNet()
        self.fc = nn.Linear(512*2, num_classes)

    def forward(self, crop_seq, full_seq):
        feat1 = self.stream1(crop_seq)
        feat2 = self.stream2(full_seq)
        fused = torch.cat((feat1, feat2), dim=1)
        return self.fc(fused)


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = TwoStreamNet(num_classes=10).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

for epoch in range(epochs):
    model.train()
    for (crop_seq, full_seq, labels) in train_loader:
        crop_seq, full_seq, labels = crop_seq.to(device), full_seq.to(device), labels.to(device)

        outputs = model(crop_seq, full_seq)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1} Loss: {loss.item():.4f}")


In [ ]:
 MobileNetV2 + Transfer Learning (Frame-based) version in PyTorch.
This will:

Extract frames from videos

Fine-tune MobileNetV2 for binary classification (Aggressive / Not Aggressive)

Average predictions from 3–5 frames per video